In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

import plotly.graph_objects as go

from strategy import Strategy

In [127]:
from gym.core import Env
from gym import spaces

POSITION_LIMIT = 200
TRANSACTION_FEE = 0.0001
FEE_IN_BIPS = 4.0
PERCENTAGE_EXCHAGNE_VOLUME = 0.03
INITIAL_POSITION = 0
ROWS_NEEDED = 2*24*60

class StockEnv(Env):
    def __init__(self, window_size = 3 * 30 * 24 * 60, start = None, end = None, verbose=True, data_path='./data/train_data.pickle'):
        super(StockEnv, self).__init__()
        
        self.start = start
        self.end = end
        
        self.training_data = pd.read_pickle(data_path)
        self.training_data['num_idx'] = list(range(self.training_data.shape[0]))
        self.window_size = window_size
        
        self.action_space = spaces.Box(-POSITION_LIMIT, POSITION_LIMIT, shape=(1,))
        
        self.verbose = verbose
        self.logs = []
        
        self.reset()

    def step(self, action):
        #todo: turbulence
        
        current_observation = self.training_data.iloc[self.train_idx]
        
        self.train_idx += 1
        current_date = current_observation.name
        
        current_volume = current_observation['volume']
        current_price = current_observation['price']
        
        self.price_history.append(current_price)
        
        if current_date >= self.end:
            self.done = True
            
        current_position = self.state[0]
        
        target_position = min(POSITION_LIMIT, max(-POSITION_LIMIT, action[0]))
        
        max_available_volume = PERCENTAGE_EXCHAGNE_VOLUME * current_volume
        
        volume_matched = min(max_available_volume, abs(target_position - current_position))
        
        direction = 1.0 if target_position > current_position else -1.0
        new_position = current_position + direction * volume_matched
        
        transaction_cost = direction * volume_matched * current_price
        fee = abs(transaction_cost) * TRANSACTION_FEE * FEE_IN_BIPS
        
        new_usd_cash = self.cash - transaction_cost - fee
        self.cash = new_usd_cash
            
        self.reward = new_usd_cash + new_position * current_price
        
        return self.state, self.reward, self.done, {}

    def reset(self):
        
        if self.start is None and self.end is None:
            self.start = np.random.choice(self.training_data.index[:-self.window_size])
            self.end = self.training_data.iloc[self.training_data.index.get_loc(self.start) + self.window_size].name
        
        # todo: decide on the state
        self.state = [0] * 1
        
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(len(self.state), ))
        
        self.done = False
        
        self.reward = 0
#         self.turbulence = 0
#         self.seed = 42
    
        self.fees = 0
        self.trades = 0
        self.cash = 0
        
        self.price_history = []
        
        #todo: maybe shift it by ROWS_NEEDED to avoid zeros at the beggining
        self.train_idx = self.training_data[self.training_data.index >= self.start]['num_idx'][0]
        
        return self.state
    
    def render(self, mode='human', close=False):
        return self.state
    
    def close(self):
#         del self.training_data
        pass
    

In [46]:
StockEnv()

In [47]:
train_data = pd.read_pickle('./data/train_data.pickle')

In [48]:
display(train_data)

,price,volume
2018-01-01 00:00:00,7889.17,24.247324
2018-01-01 00:01:00,7934.37,37.126923
2018-01-01 00:02:00,8055.16,33.290021
2018-01-01 00:03:00,7870.31,53.375096
2018-01-01 00:04:00,7749.08,24.575963
...,...,...
2021-12-31 23:55:00,16832.76,39.870865
2021-12-31 23:56:00,16828.93,13.840201
2021-12-31 23:57:00,16817.76,22.845215
2021-12-31 23:58:00,16796.24,85.997617


In [57]:
agg_window = 60 * 1
train_data['group'] = [ x // agg_window for x in range(train_data.shape[0] ) ]

In [58]:
display(train_data)

,price,volume,group
2018-01-01 00:00:00,7889.17,24.247324,0
2018-01-01 00:01:00,7934.37,37.126923,0
2018-01-01 00:02:00,8055.16,33.290021,0
2018-01-01 00:03:00,7870.31,53.375096,0
2018-01-01 00:04:00,7749.08,24.575963,0
...,...,...,...
2021-12-31 23:55:00,16832.76,39.870865,34912
2021-12-31 23:56:00,16828.93,13.840201,34912
2021-12-31 23:57:00,16817.76,22.845215,34912
2021-12-31 23:58:00,16796.24,85.997617,34912


In [59]:
data = train_data.reset_index().groupby('group').agg({
    'price': [('high', 'max'), ('low', 'min'), ('open', 'first'), ('close', 'last')],
    'index': [('index', 'last')]
})

In [60]:
data.columns = data.columns.droplevel(0)

In [61]:
data

,high,low,open,close,index
group,,,,,
0,8055.16000,7552.68,7889.17,7635.72,2018-01-01 00:59:00
1,7824.96000,7196.80,7651.00,7196.80,2018-01-01 01:59:00
2,7578.20000,7193.81,7392.00,7329.76,2018-01-01 02:59:00
3,7802.92000,7296.36,7357.16,7471.44,2018-01-01 03:59:00
4,7775.46000,7303.88,7485.08,7629.68,2018-01-01 04:59:00
...,...,...,...,...,...
34908,16955.71000,16708.71,16858.99,16808.93,2021-12-31 20:28:00
34909,16829.83000,16689.39,16798.60,16804.21,2021-12-31 21:28:00
34910,16906.72946,16763.82,16809.33,16856.51,2021-12-31 22:28:00


In [68]:
sample = data.sample(1000)

In [ ]:
class CandleStick:
    def __init__(self, window_size=10):
        self.window_size = window_size
        
        self.high = None
        self.low = None
        self.open = None
        self.close = None
        
    def step(self, new_value):
        pass

In [89]:
class SimpleMovingAverage:
    def __init__(self, window_size, value=0, method='fast'):
        self.value = value
        self.window_size = window_size
        
        if method == 'fast':
            self.step = self.fast_step
        elif method == 'precice':
            self.step = self.precice_step
            self.history = [self.value]
        else:
            raise 'ValueError'
        
#     def step(self, new_value):
#         self.value = self.value
        
    def fast_step(self, new_value):
        self.value = (self.value * (self.window_size-1) + new_value) / self.window_size
    
    def precice_step(self, new_value):
        self.history.append(new_value)
        self.value = np.mean(self.history[-self.window_size:])
    
    def get(self):
        return self.value

In [90]:
xs = np.sin(np.arange(0, 2 * np.pi, 0.001))

In [126]:
class ExponentialMovingAverage:
    def __init__(self, alpha, starting_value=0):
        self.value = starting_value
        self.alpha = alpha
        
    def step(self, new_value):
        self.value = self.value * self.alpha + (1 - self.alpha) * new_value
        
    def get(self):
        return self.value

In [74]:
class MovingAverageConvergenceDivergence:
    def __init__(self, short_window = 10, long_window = 20):
        self.value = None
        self.short_window = short_window
        self.long_window = long_window
        self.history = []
    
    def step(self, new_value):
        self.history.append(new_value)
        
        long_window = self.history[-self.long_window:]
        short_window = long_window[-self.short_window:]
        
        
        
        pass
    
    def get(self):
        return self.value

In [75]:
class RelativeStrengthIndicator:
    def __init__(self):
        self.value = None
    
    def step(self):
        pass
    
    def get(self):
        return self.value

In [69]:
fig = go.Figure(data=[go.Candlestick(x=sample['index'],
                open=sample['open'],
                high=sample['high'],
                low=sample['low'],
                close=sample['close'])])

In [70]:
fig.show()